## Background
We assume the system is $x \in R^n$ and $x[t+1] = Ax[t] + Bu[t]$ where $u[t]=K\hat{x}[t]$ is computed from the neural-network perceived states $\hat{x}[t] \approx x[t]$.


## Multiplicative noise model



We model noise by $\hat{x}[t] = Ex[t]$ where $E$ is a diagonal matrix that is close to the identity matrix. Specifically, given a small $\epsilon_i$ such that $\hat{x}_i[t] = (1+\epsilon_i)x_i[t]$,
<!-- given $\lambda_i \approx 1$ such that $\hat{x}_i[t] \in (\frac{x_i[t]}{\lambda_i}, x_i[t]\lambda_i)$ -->
$$
E = \begin{bmatrix}
    1+\epsilon_1 & & \huge0 \\
    & \ddots &  \\
    \huge0 & & 1+\epsilon_n
\end{bmatrix},
$$
and the system model becomes
$$
x[t+1] = Ax[t] + BK(Ex[t]) \\
       = (A+BKE)x[t].
$$

We use a special notation $E_i$ to denote when only **one** state, $x_i$, is affected by uncertainty and all other states are assumed to be perfectly sensed.
$$
E_i = \begin{bmatrix}
    I_{i-1} & & \huge0 \\
    & 1+\epsilon_i &  \\
    \huge0 & & I_{n-i}
\end{bmatrix}.
$$

For example, when $A=
\begin{bmatrix}
a_{11} & a_{11} \\
a_{21} & a_{22}
\end{bmatrix}
$, $B=\begin{bmatrix}
    b_1 \\ b_2
\end{bmatrix}$, and $K = \begin{bmatrix}
    k_1 &  k_2
\end{bmatrix}$,
$$
A+BKE_1 =
\begin{bmatrix}
a_{11}+b_1k_1 (1+\epsilon_1) & a_{11}+b_1k_2 \\
a_{21}+b_2k_1 (1+\epsilon_1) & a_{22}+b_2k_2
\end{bmatrix},
$$
and
$$
A+BKE_2 =
\begin{bmatrix}
a_{11}+b_1k_1 & a_{11}+b_1k_2 (1+\epsilon_2) \\
a_{21}+b_2k_1 & a_{22}+b_2k_2 (1+\epsilon_2)
\end{bmatrix}.
$$

## Computing sensitivity with multiplicative noise model


The key of the technique is to model the system as 
$$x[t+1] = A_\phi x[t]$$
and model the uncertainty in a way such that the system with uncertainty is represented by 
$$x[t+1] = A_\phi' x[t]$$
where $A_\phi' = A_\phi + \epsilon Y + O(\epsilon^2)$. 
Then using $A_\phi$ and $Y$ as inputs, we use the prior technique to compute the sensitivity of maximum singular value of $A_\phi'$ relative to $\epsilon$. Using the previous example, we see that 
$$
\begin{align*}
A_\phi 
&= A+BK \\
&= \begin{bmatrix}
    a_{11}+b_1k_1 & a_{11}+b_1k_2 \\
    a_{21}+b_2k_1 & a_{22}+b_2k_2
\end{bmatrix},
\end{align*}
$$
And for a perturbation in $x_1$, the resulting $A_\phi'$ becomes
$$
\begin{align*}
A_\phi' 
&= A+BKE_1 \\
&=\begin{bmatrix}
    a_{11}+b_1k_1 (1+\epsilon_1) & a_{11}+b_1k_2 \\
    a_{21}+b_2k_1 (1+\epsilon_1) & a_{22}+b_2k_2
\end{bmatrix} \\
&= \begin{bmatrix}
    a_{11}+b_1k_1 & a_{11}+b_1k_2 \\
    a_{21}+b_2k_1 & a_{22}+b_2k_2
\end{bmatrix} +
\epsilon_1 \begin{bmatrix}
    b_1k_1 & 0 \\
    b_2k_1 & 0
\end{bmatrix} \\
&= A_\phi + \epsilon_1 Y_1
\end{align*}
$$
where $Y_1 = \begin{bmatrix}
    b_1k_1 & 0 \\
    b_2k_1 & 0
\end{bmatrix}$. Similarly, $Y_2 =
\begin{bmatrix}
    0 & b_1k_2 \\
    0 & b_2k_2
\end{bmatrix}$.

More generally, for arbitrary $A$, $B$ and $K$, $A_\phi = A+BK$ and
$Y_i = \begin{bmatrix}
    & b_1k_i & \\
    \huge 0 & \vdots & \huge 0 \\
    & b_nk_i & \\
\end{bmatrix}.$

Similarly, for open-loop systems, $A_\phi = A$ and
$Y_i = \begin{bmatrix}
    & a_{1i} & \\
    \huge 0 & \vdots & \huge 0 \\
    & a_{ni} & \\
\end{bmatrix}.$

In [1]:
import numpy as np
import control as ctrl
from Benchmarks import sys_variables
from OrderUncertainties import *

def get_sensitivity(A_phi, Y):
    mat = OrdUnc(A_phi)
    if mat.determineCase() == 1:
        return mat.distinctPos(Y)
    else:
        return mat.multSig(Y)

def mult_sensitivity(A, B, K=None):
    if K is not None:
        A_phi = A + np.matmul(B, K)
        Y = np.matmul(B, K)
    else:
        A_phi = A
        Y = A

    sensitivity = []
    for i in range(Y.shape[1]):
        Y_i = np.zeros(Y.shape)
        Y_i[:, i] = Y[:, i]
        sensitivity.append(get_sensitivity(A_phi, Y_i))

    return sensitivity

round2g = lambda x: float(f"{x:.2g}")

In [14]:
res = {}
for (name, bench) in sys_variables.items():
    K = ctrl.dlqr(bench.sysd, np.eye(bench.nx), np.eye(bench.nu))[0]
    res[name] = list(map(round2g, mult_sensitivity(bench.sysd.A, bench.sysd.B, K)))
res

{'RC': [0.03, 0.0029],
 'F1': [0.13, 0.35],
 'DC': [1.3e-05, 0.016],
 'CS': [0.54, 0.075, 0.52, 0.066],
 'EW': [320.0, 0.039],
 'C1': [2e-05],
 'CC': [0.0025, 0.008, 0.049],
 'D5': [0.082, 0.22, 0.045, 0.17, 0.08]}

In [16]:
bench = sys_variables['D5']
print(bench.sysd.A, np.ones((bench.sysd.A.shape[0], 1)))
K = ctrl.dlqr(bench.sysd.A, np.ones((bench.sysd.A.shape[0], 1)), np.eye(bench.nx), np.eye(bench.nu))[0]
list(map(round2g, mult_sensitivity(bench.sysd.A, bench.sysd.B, K)))

[[ 0.70928684 -0.0927456   0.13492716 -0.01850984 -0.14914274]
 [-0.21832911  0.17897517 -0.09644953 -0.39045726  0.41558154]
 [ 0.68196065  1.24570196  0.99419551  0.83227819 -0.65418901]
 [ 0.09193552  0.39630738  0.04493402  1.0934956  -0.34267877]
 [-0.42660553 -0.95573144  0.13861052 -0.44067533  0.98383295]] [[1.]
 [1.]
 [1.]
 [1.]
 [1.]]


[0.037, 0.046, 0.026, 0.022, 0.0072]

## Additive noise model

In the additive noise model, the noise $w$ is not dependent on the current state $x[t]$. *i.e.*, $\hat{x}_i[t] = x_i[t] + w_i$.
$$
\begin{align*}
x[t+1] &= Ax[t] + BK(x[t]+w) \\
    &= (A+BK)x[t] + BKw
\end{align*}
$$

Since
$
BK =
\begin{bmatrix} b_1k_1 & b_1k_2\\ 
b_2k_1 & b_2k_2 \end{bmatrix},
$
The two parts of the expressions are
$$
(A+BK)x=
\begin{bmatrix} (a_{11}+b_1k_1)x_1+(a_{12}+b_1k_2)x_2 \\ 
(a_{21}+b_2k_1)x_1+(a_{22}+b_2k_2)x_2 \end{bmatrix},
$$
and
$$
BKw =
\begin{bmatrix} b_1k_1w_1+b_1k_2w_2 \\
 b_2k_1w_1+b_2k_2w_2 \end{bmatrix}
$$

Let $w = \begin{bmatrix}
    1 \\ 0
\end{bmatrix}$, then
$$
BKw =
\begin{bmatrix} b_1k_1 \\
 b_2k_1 \end{bmatrix}
$$

$$
(A+BK)x+BKw=
\begin{bmatrix} (a_{11}+b_1k_1)x_1+(a_{12}+b_1k_2)x_2+b_1k_1w_1+b_1k_2w_2 \\
 (a_{21}+b_2k_1)x_1+(a_{22}+b_2k_2)x_2+b_2k_1w_1+b_2k_2w_2 \end{bmatrix}
$$

## Sensitivity with additive noise model

Our method requires the perturbation to be reflected in the augmented matrix. One way to achieve this is to add a "fake" state in $x$ and modify $A$, $B$ and $K$ matrices accordingly:

$$
\bar{x} = \begin{bmatrix} x_1 \\ x_2 \\ 1 \end{bmatrix} \\
\bar{A} = \begin{bmatrix}
    a_{11} & a_{12} & 0 \\
    a_{21} & a_{22} & 0 \\
    0 & 0 & 1 
\end{bmatrix} \\
\bar{B}\bar{K} = \begin{bmatrix}
    b_1k_1 & b_1k_2 & 0 \\
    b_2k_1 & b_2k_2 & 0 \\
    0 &    0 &    0 
\end{bmatrix}
$$

Recall that $x[t+1] = (\bar{A}+\bar{B}\bar{K})x[t] + BKw$, then
$$
w' = \begin{bmatrix}
    0 & 0 & b_1k_1w_1+b_1k_2w_2 \\
    0 & 0 & b_2k_1w_1+b_2k_2w_2 \\
    0 & 0 & 0
\end{bmatrix} \\
w'_1 = \begin{bmatrix}
    0 & 0 & b_1k_1 \\
    0 & 0 & b_2k_1 \\
    0 & 0 & 0
\end{bmatrix} \\
\bar{A}+\bar{B}\bar{K}+w' =
\begin{bmatrix}
    a_{11}+b_1k_1 & a_{11}+b_1k_2 & b_1k_1w_1+b_1k_2w_2 \\
    a_{21}+b_2k_1 & a_{22}+b_2k_2 & b_2k_1w_1+b_2k_2w_2 \\
    0 & 0 & 1
\end{bmatrix} \\
x[t+1] = (\bar{A}+\bar{B}\bar{K}+w')x[t] = \begin{bmatrix}
    (a_{11}+b_1k_1)x_1+(a_{12}+b_1k_2)x_2+b_1k_1w_1+b_1k_2w_2 \\
    (a_{21}+b_2k_1)x_1+(a_{22}+b_2k_2)x_2+b_2k_1w_1+b_2k_2w_2 \\
    1
\end{bmatrix}
$$
is the same formulation of before with the uncertainty modeled within $A_\phi' = \bar{A}+\bar{B}\bar{K}+w'$

$$
A_\phi = \bar{A}+\bar{B}\bar{K} \\
Y_i = w'_i
$$

For example,
$$
\begin{align*}
A\phi' &= A_\phi + w_1 Y_1 \\
&= \bar{A}+\bar{B}\bar{K} + w_1 \begin{bmatrix}
    0 & 0 & b_1k_1 \\
    0 & 0 & b_2k_1 \\
    0 & 0 & 0
\end{bmatrix} \\
&= \begin{bmatrix}
    a_{11}+b_1k_1 & a_{11}+b_1k_2 & b_1k_1w_1 \\
    a_{21}+b_2k_1 & a_{22}+b_2k_2 & b_2k_1w_1 \\
    0 & 0 & 1
\end{bmatrix}
\end{align*}
$$

x <- R^n
w <- R^n, only 1 element in w is non-zero
$$
Y =
\begin{bmatrix}
0 & 0 & 1 \\
0 & 0 & 1 \\
\end{bmatrix}
$$

$$
\begin{bmatrix} a_{11}+b_1k_1 & a_{12}+b_1k_2+l\\ 
a_{21}+b_2k_1 & a_{22}+b_2k_2+l \end{bmatrix}
$$

In [9]:
def addi_sensitivity(A, B, K=None):
    A_bar = np.pad(A, ((0, 1), (0, 1)))
    A_bar[-1, -1] = 1
    B_bar = np.pad(B, ((0, 1), (0, 0)))
    if K is not None:
        K_bar = np.pad(K, ((0, 0), (0, 1)))
        A_phi = A_bar + np.matmul(B_bar, K_bar)
        Y = np.matmul(B_bar, K_bar)
    else:
        A_phi = A_bar
        Y = np.pad(np.eye(A.shape[0]), ((0, 1), (0, 1)))

    sensitivity = []
    for i in range(Y.shape[1]):
        Y_i = np.zeros(Y.shape)
        Y_i[:, -1] = Y[:, i]
        if (i == 1):
            print(A_phi, Y_i, get_sensitivity(A_phi, Y_i), sep='\n')
        sensitivity.append(get_sensitivity(A_phi, Y_i))

    return sensitivity[:-1]


In [42]:
bench = sys_variables['F1']
K = ctrl.lqr(bench.sysd, np.eye(bench.nx), np.eye(bench.nu))[0]
addi_sensitivity(bench.sysd.A, bench.sysd.B, K)

[[0.         0.         0.01988499]
 [0.         0.         0.30592288]
 [0.         0.         0.        ]]
[[0.         0.         0.02704568]
 [0.         0.         0.41608744]
 [0.         0.         0.        ]]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


[0.006947447552822857, 0.009449262727179473, 0.0]

In [10]:
res = {}
for (name, bench) in sys_variables.items():
    K = ctrl.lqr(bench.sysd, np.eye(bench.nx), np.eye(bench.nu))[0]
    res[name] = list(map(round2g, addi_sensitivity(bench.sysd.A, bench.sysd.B, K)))
res

[[0.91874595 0.03095522 0.        ]
 [0.00380713 0.98616049 0.        ]
 [0.         0.         1.        ]]
[[0.00000000e+00 0.00000000e+00 1.22423103e-02]
 [0.00000000e+00 0.00000000e+00 2.48567405e-05]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]]
5.873117857938395e-06
[[1.01988499 0.15704568 0.        ]
 [0.30592288 1.41608744 0.        ]
 [0.         0.         1.        ]]
[[0.         0.         0.02704568]
 [0.         0.         0.41608744]
 [0.         0.         0.        ]]
0.009449262727179502
[[ 8.18729639e-01  1.79070124e-02  0.00000000e+00]
 [-1.06604336e-04  9.76667847e-01  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[0.         0.         0.0001497 ]
 [0.         0.         0.01588205]
 [0.         0.         0.        ]]
0.000969031904743041
[[ 9.98144847e-01  1.91005471e-02  2.50365834e-03  2.98242303e-04
   0.00000000e+00]
 [-1.62653556e-01  9.20544771e-01  2.00552303e-01  2.43290310e-02
   0.00000000e+00]
 [-2.39348858e-03 -3.19856002e-

{'RC': [1.5e-05, 5.9e-06],
 'F1': [0.0069, 0.0094],
 'DC': [1.5e-05, 0.00097],
 'CS': [0.051, 0.022, 0.098, 0.021],
 'EW': [0.00027, 3e-06],
 'C1': [0.0],
 'CC': [0.00063, 0.0078, 0.0079],
 'D5': [0.0, 0.0, 0.0, 0.0, 0.0]}

In [11]:
bench = sys_variables['D5']
list(map(round2g, addi_sensitivity(bench.sysd.A, bench.sysd.B)))

[[ 0.70928684 -0.0927456   0.13492716 -0.01850984 -0.14914274  0.        ]
 [-0.21832911  0.17897517 -0.09644953 -0.39045726  0.41558154  0.        ]
 [ 0.68196065  1.24570196  0.99419551  0.83227819 -0.65418901  0.        ]
 [ 0.09193552  0.39630738  0.04493402  1.0934956  -0.34267877  0.        ]
 [-0.42660553 -0.95573144  0.13861052 -0.44067533  0.98383295  0.        ]
 [ 0.          0.          0.          0.          0.          1.        ]]
[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
0.0


[0.35, 0.0, 0.23, 0.52, 0.32]

[0.45, 0.29, 0.48, 0.68, 0.27]